# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   548.100827  0.607465  1234  548.708292  547.493362  1097.416583   
6   255.828612  0.306950  1234  256.135562  255.521662   512.271125   
7   174.558422 -0.818384  1234  173.740038  175.376806   347.480075   
8   555.388669 -0.220590  1234  555.168079  555.609259  1110.336158   
9   248.132717  0.078322  1234  248.211039  248.054395   496.422079   
10  914.825915 -0.230949  1234  914.594966  915.056863  1829.189933   
11  792.967891  0.463819  1234  793.431710  792.504072  1586.863419   
12   59.071168  0.530676  1234   59.601844   58.540493   119.203688   
13  446.451443  0.932038  1234  447.383482  445.519405   894.766963   
14  276.377548  0.070406  1234  276.447954  276.307142   552.895907   

              f  
5   1094.986723  
6    511.043324  
7    350.753613  
8   1111.218518  
9    496.108790  
10  1830.113727  
11  1585.008144  
12   117.080986  
13   891.038810  
14   552.614285

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -548.708292   547.493362
6   -256.135562   255.521662
7   -173.740038   175.376806
8   -555.168079   555.609259
9   -248.211039   248.054395
10  -914.594966   915.056863
11  -793.431710   792.504072
12   -59.601844    58.540493
13  -447.383482   445.519405
14  -276.447954   276.307142

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -548.708292   547.493362
6   -256.135562   255.521662
7   -173.740038   175.376806
8   -555.168079   555.609259
9   -248.211039   248.054395
10  -914.594966   915.056863
11  -793.431710   792.504072
12   -59.601844    58.540493
13  -447.383482   445.519405
14  -276.447954   276.307142

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-548.70829169,  547.49336164],
       [-256.13556234,  255.52166202],
       [-173.74003774,  175.3768064 ],
       [-555.16807905,  555.60925896],
       [-248.21103929,  248.05439511],
       [-914.59496632,  915.05686333],
       [-793.43170963,  792.50407206],
       [ -59.60184392,   58.54049279],
       [-447.38348154,  445.51940498],
       [-276.44795354,  276.30714238]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5    1095.416583  -1094.986723
6     510.271125   -511.043324
7     345.480075   -350.753613
8    1108.336158  -1111.218518
9     494.422079   -496.108790
10   1827.189933  -1830.113727
11   1584.863419  -1585.008144
12    117.203688   -117.080986
13    892.766963   -891.038810
14    550.895907   -552.614285

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf